<a href="https://colab.research.google.com/github/vvamsi91/RTML_AS5/blob/main/RTML_AS534.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
english_to_french = [
    ("I am cold", "J'ai froid"),
    ("You are tired", "Tu es fatigué"),
    ("He is hungry", "Il a faim"),
    ("She is happy", "Elle est heureuse"),
    ("We are friends", "Nous sommes amis"),
    ("They are students", "Ils sont étudiants"),
    ("The cat is sleeping", "Le chat dort"),
    ("The sun is shining", "Le soleil brille"),
    ("We love music", "Nous aimons la musique"),
    ("She speaks French fluently", "Elle parle français couramment"),
    ("He enjoys reading books", "Il aime lire des livres"),
    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),
    ("The movie starts at 7 PM", "Le film commence à 19 heures"),
    ("She wears a red dress", "Elle porte une robe rouge"),
    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),
    ("He drives a blue car", "Il conduit une voiture bleue"),
    ("They visit museums often", "Ils visitent souvent des musées"),
    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),
    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),
    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),
    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),
    ("They travel around the world", "Ils voyagent autour du monde"),
    ("The book is on the table", "Le livre est sur la table"),
    ("She dances gracefully", "Elle danse avec grâce"),
    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),
    ("He works hard every day", "Il travaille dur tous les jours"),
    ("They speak different languages", "Ils parlent différentes langues"),
    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),
    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),
    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),
    ("The dog barks loudly", "Le chien aboie bruyamment"),
    ("He sings beautifully", "Il chante magnifiquement"),
    ("They swim in the pool", "Ils nagent dans la piscine"),
    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),
    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),
    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),
    ("He paints landscapes", "Il peint des paysages"),
    ("They laugh at the joke", "Ils rient de la blague"),
    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),
    ("She runs in the park", "Elle court dans le parc"),
    ("We travel by train", "Nous voyageons en train"),
    ("He writes a letter", "Il écrit une lettre"),
    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),
    ("The baby cries", "Le bébé pleure"),
    ("She studies hard for exams", "Elle étudie dur pour les examens"),
    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),
    ("He fixes the car", "Il répare la voiture"),
    ("They drink coffee in the morning", "Ils boivent du café le matin"),
    ("The sun sets in the evening", "Le soleil se couche le soir"),
    ("She dances at the party", "Elle danse à la fête"),
    ("We play music at the concert", "Nous jouons de la musique au concert"),
    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),
    ("They study French grammar", "Ils étudient la grammaire française"),
    ("The rain falls gently", "La pluie tombe doucement"),
    ("She sings a song", "Elle chante une chanson"),
    ("We watch a movie together", "Nous regardons un film ensemble"),
    ("He sleeps deeply", "Il dort profondément"),
    ("They travel to Paris", "Ils voyagent à Paris"),
    ("The children play in the park", "Les enfants jouent dans le parc"),
    ("She walks along the beach", "Elle se promène le long de la plage"),
    ("We talk on the phone", "Nous parlons au téléphone"),
    ("He waits for the bus", "Il attend le bus"),
    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),
    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),
    ("She dreams of flying", "Elle rêve de voler"),
    ("We work in the office", "Nous travaillons au bureau"),
    ("He studies history", "Il étudie l'histoire"),
    ("They listen to the radio", "Ils écoutent la radio"),
    ("The wind blows gently", "Le vent souffle doucement"),
    ("She swims in the ocean", "Elle nage dans l'océan"),
    ("We dance at the wedding", "Nous dansons au mariage"),
    ("He climbs the mountain", "Il gravit la montagne"),
    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),
    ("The cat meows loudly", "Le chat miaule bruyamment"),
    ("She paints a picture", "Elle peint un tableau"),
    ("We build a sandcastle", "Nous construisons un château de sable"),
    ("He sings in the choir", "Il chante dans le chœur")
]

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

# Define tokens to signify the start and end of sequences
SOS_token, EOS_token = 0, 1

# Assume english_to_french dataset exists; create reversed pairs for training
french_to_english = [(french, english) for english, french in english_to_french]

# Map all unique words to indices, including special tokens
word_to_index = {"SOS": SOS_token, "EOS": EOS_token}
for source, target in french_to_english:
    for word in source.split() + target.split():
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)

# Dataset class for translation using indexed word representation
class TranslationDataset(Dataset):
    """Manages French to English translation pairs."""
    def __init__(self, data, index_map):
        self.data = data
        self.index_map = index_map

    def __len__(self):
        # Return the count of data pairs
        return len(self.data)

    def __getitem__(self, idx):
        # Convert sentences to indices, append EOS, and return as tensors
        source, target = self.data[idx]
        source_indices = [self.index_map[word] for word in source.split()] + [EOS_token]
        target_indices = [self.index_map[word] for word in target.split()] + [EOS_token]
        return torch.tensor(source_indices, dtype=torch.long), torch.tensor(target_indices, dtype=torch.long)

# Example DataLoader setup for training
# dataset = TranslationDataset(french_to_english, word_to_index)
# loader = DataLoader(dataset, batch_size=2, shuffle=True)


In [ ]:
def collate_batch(batch):
    inputs, targets = zip(*batch)
    padded_inputs = pad_sequence(inputs, batch_first=True, padding_value=EOS_token)
    padded_targets = pad_sequence(targets, batch_first=True, padding_value=EOS_token)
    return padded_inputs, padded_targets

# Transformer-based model for language translation tasks
class TranslationModel(nn.Module):
    """A Transformer model for sequence-to-sequence learning."""
    def __init__(self, vocab_size, hidden_size, num_layers=4, num_heads=8, dropout=0.1):
        super(TranslationModel, self).__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.transformer = nn.Transformer(
            d_model=hidden_size,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=hidden_size * 4,
            dropout=dropout,
            batch_first=True  # Specifies that the batch dimension is the first dimension
        )
        self.final_layer = nn.Linear(hidden_size, vocab_size)

    def forward(self, src, tgt):
        src_embedded = self.embed(src)
        tgt_embedded = self.embed(tgt)
        tgt_mask = self.transformer.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)
        transformer_out = self.transformer(src_embedded, tgt_embedded, tgt_mask=tgt_mask)
        final_output = self.final_layer(transformer_out)
        return final_output

In [ ]:
def train_and_evaluate(model, dataloader, optimizer, criterion, epochs, device):
    """Train and evaluate the model across a specified number of epochs."""
    for epoch in range(epochs+1):
        model.train()  # Set model to training mode
        total_loss, total_correct, total_samples = 0, 0, 0

        # Training phase
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs, targets[:, :-1])
            outputs_flat = outputs.view(-1, outputs.size(-1))
            targets_flat = targets[:, 1:].contiguous().view(-1)

            # Calculate loss excluding the EOS token
            valid_targets_mask = targets_flat != EOS_token
            loss = criterion(outputs_flat[valid_targets_mask], targets_flat[valid_targets_mask])
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, preds = torch.max(outputs_flat, 1)
            correct_preds = (preds == targets_flat) & valid_targets_mask
            total_correct += correct_preds.sum().item()
            total_samples += valid_targets_mask.sum().item()

        # Logging the training results
        avg_loss = total_loss / len(dataloader)
        accuracy = total_correct / total_samples if total_samples > 0 else 0

        # Evaluation phase with model in evaluation mode
        model.eval()
        eval_loss, eval_correct, eval_samples = 0, 0, 0
        with torch.no_grad():
            for inputs, targets in dataloader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs, targets[:, :-1])
                outputs_flat = outputs.view(-1, outputs.size(-1))
                targets_flat = targets[:, 1:].contiguous().view(-1)

                valid_targets_mask = targets_flat != EOS_token
                loss = criterion(outputs_flat[valid_targets_mask], targets_flat[valid_targets_mask])
                eval_loss += loss.item()
                _, preds = torch.max(outputs_flat, 1)
                correct_preds = (preds == targets_flat) & valid_targets_mask
                eval_correct += correct_preds.sum().item()
                eval_samples += valid_targets_mask.sum().item()

        # Calculate evaluation metrics
        eval_avg_loss = eval_loss / len(dataloader)
        eval_accuracy = eval_correct / eval_samples if eval_samples > 0 else 0

        # Print results periodically
        if epoch % 5 == 0 :
            print(f'Epoch {epoch}: Train Loss = {avg_loss:.4f}, Accuracy = {accuracy:.4f}, '
                  f'Eval Loss = {eval_avg_loss:.4f}, Eval Accuracy = {eval_accuracy:.4f}')



In [ ]:
# Prepare the environment and model parameters
vocab_size = len(word_to_index)
hidden_size = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TranslationModel(vocab_size, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=word_to_index["EOS"])

# Initialize DataLoader for handling the dataset
dataset = TranslationDataset(english_to_french, word_to_index)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_batch)

# Execute training and evaluation for the model
train_and_evaluate(model, dataloader, optimizer, criterion, 50, device)

Epoch 0: Train Loss = 5.9007, Accuracy = 0.0134, Eval Loss = 5.6278, Eval Accuracy = 0.0638
Epoch 5: Train Loss = 5.0578, Accuracy = 0.1074, Eval Loss = 4.8524, Eval Accuracy = 0.0973
Epoch 10: Train Loss = 4.4069, Accuracy = 0.1477, Eval Loss = 4.1166, Eval Accuracy = 0.2081
Epoch 15: Train Loss = 3.7548, Accuracy = 0.3121, Eval Loss = 3.3862, Eval Accuracy = 0.3591
Epoch 20: Train Loss = 3.2297, Accuracy = 0.3893, Eval Loss = 2.7517, Eval Accuracy = 0.5470
Epoch 25: Train Loss = 2.6662, Accuracy = 0.5940, Eval Loss = 2.2097, Eval Accuracy = 0.7282
Epoch 30: Train Loss = 2.1611, Accuracy = 0.7148, Eval Loss = 1.6601, Eval Accuracy = 0.8456
Epoch 35: Train Loss = 1.7365, Accuracy = 0.8389, Eval Loss = 1.2626, Eval Accuracy = 0.9430
Epoch 40: Train Loss = 1.4150, Accuracy = 0.8725, Eval Loss = 0.9720, Eval Accuracy = 0.9832
Epoch 45: Train Loss = 1.0852, Accuracy = 0.9664, Eval Loss = 0.7221, Eval Accuracy = 0.9966
Epoch 50: Train Loss = 0.9017, Accuracy = 0.9799, Eval Loss = 0.5386, Ev

In [ ]:
# Setup
vocab_size = len(word_to_index)
hidden_size = 64
model = TranslationModel(vocab_size, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss(ignore_index=word_to_index["EOS"])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = TranslationDataset(french_to_english, word_to_index)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_batch)

# Train and evaluate
train_and_evaluate(model, dataloader, optimizer, criterion, 100, device)

Epoch 0: Train Loss = 6.0108, Accuracy = 0.0037, Eval Loss = 5.7182, Eval Accuracy = 0.0769
Epoch 5: Train Loss = 5.3420, Accuracy = 0.0916, Eval Loss = 5.1597, Eval Accuracy = 0.0952
Epoch 10: Train Loss = 4.8479, Accuracy = 0.1099, Eval Loss = 4.6396, Eval Accuracy = 0.1319
Epoch 15: Train Loss = 4.3997, Accuracy = 0.1905, Eval Loss = 4.0844, Eval Accuracy = 0.2198
Epoch 20: Train Loss = 3.9467, Accuracy = 0.2308, Eval Loss = 3.5940, Eval Accuracy = 0.3736
Epoch 25: Train Loss = 3.6494, Accuracy = 0.3846, Eval Loss = 3.2519, Eval Accuracy = 0.4689
Epoch 30: Train Loss = 3.2580, Accuracy = 0.4396, Eval Loss = 2.8375, Eval Accuracy = 0.6117
Epoch 35: Train Loss = 2.9553, Accuracy = 0.5238, Eval Loss = 2.5574, Eval Accuracy = 0.6740
Epoch 40: Train Loss = 2.6735, Accuracy = 0.6337, Eval Loss = 2.2113, Eval Accuracy = 0.7436
Epoch 45: Train Loss = 2.4002, Accuracy = 0.7033, Eval Loss = 1.9001, Eval Accuracy = 0.8059
Epoch 50: Train Loss = 2.1528, Accuracy = 0.7289, Eval Loss = 1.6805, Ev